# Used to collect data

In [1]:
import tweepy
import cnfg
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob
from pymongo import MongoClient
from tweepy import Stream
from tweepy.streaming import StreamListener
import json

pd.set_option('display.max_colwidth', 200)

In [2]:
config = cnfg.load(".twitter_config")

auth = tweepy.OAuthHandler(config["consumer_key"],
                           config["consumer_secret"])
auth.set_access_token(config["access_token"],
                      config["access_token_secret"])

api=tweepy.API(auth)

In [3]:
# Connect to monogodb server.  Make sure it's running first!
client = MongoClient('localhost', 27017)

In [4]:
client.list_database_names()

['admin', 'config', 'local', 'twitter_database']

In [5]:
# Connect to the database for the first time
twitter_db = client['twitter_database']

# Create a collection/change collection
# tweet_collection = twitter_db.tweets
# tweet_collection.drop()
tweet_collection = twitter_db.rdr2_no_retweets_2
tweet_collection.count()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


8834

In [6]:
# client.drop_database('twitter_database')

In [8]:
import pprint
from IPython.display import clear_output

class listener(StreamListener):

    def on_data(self, data):
        data = json.loads(data)
        
        # Pull the fields we want, and throw it into our mongodb database
        if data['lang'] == 'en' and 'retweeted_status' not in data:
            clear_output(wait=True)
            print(tweet_collection.count())
            tweet_document = {}
            
            pp = pprint.PrettyPrinter()
            pp.pprint(data)

            tweet_document['created_at'] = data['created_at']
            tweet_document['favorite_count'] = data['favorite_count']
            tweet_document['retweet_count'] = data['retweet_count']

            #Get full text if it is an extended tweet
            if 'extended_tweet' in data.keys():
                tweet_document['text'] = data['extended_tweet']['full_text']
            else:
                tweet_document['text'] = data['text']

            tweet_document['screen_name'] = data['user']['screen_name']

            text = TextBlob(tweet_document['text']).sentiment

            tweet_document['sentiment'] = text.polarity
            tweet_document['subjectivity'] = text.subjectivity

           # print(tweet_document)
           # print('\n')
            tweet_collection.insert_one(tweet_document)
        
        return True

    def on_error(self, status):
        print(status)

In [9]:
# regex test for filtering panda or mongo later
# import re
# pattern = re.compile(r'youtube')
# find = re.match(pattern, 'www.youtube.com')
# print(find.group())

In [16]:
twitterStream = Stream(auth, listener())
data = twitterStream.filter(track=['red dead redemption', 'rdr2', 'red dead', 'rdr', 'RDR2', 'RDR'])

47368
{'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Nov 15 01:07:33 +0000 2018',
 'display_text_range': [0, 70],
 'entities': {'hashtags': [],
              'media': [{'display_url': 'pic.twitter.com/I9QD8fjRnC',
                         'expanded_url': 'https://twitter.com/Hayden_G230/status/1062874711372120065/photo/1',
                         'id': 1062874701083508736,
                         'id_str': '1062874701083508736',
                         'indices': [71, 94],
                         'media_url': 'http://pbs.twimg.com/media/DsAW6MbVAAAYtpO.jpg',
                         'media_url_https': 'https://pbs.twimg.com/media/DsAW6MbVAAAYtpO.jpg',
                         'sizes': {'large': {'h': 750,
                                             'resize': 'fit',
                                             'w': 1334},
                                   'medium': {'h': 675,
                                              'resize': 'fit',
                         

ReadTimeoutError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.

In [81]:
twitter_db['rdr2_tweets'].find_one()

{'_id': ObjectId('5be4b92e2af8b804e27908bd'),
 'created_at': 'Thu Nov 08 22:31:05 +0000 2018',
 'favorite_count': 0,
 'retweet_count': 0,
 'text': 'Rockstar Games earned $800m for first 3 days of Red dead redemption 2. :O',
 'screen_name': 'lukasneo',
 'sentiment': 0.024999999999999998,
 'subjectivity': 0.43333333333333335}

In [70]:
# res = twitter_db['rdr2_tweets'].find().sort([('created_at',-1)])#.limit(1000)
# for i in res:
#     print(i)
# print(res)

In [82]:
res = twitter_db['rdr2_tweets'].find().sort([('created_at',-1)])#.limit(1000)
res_list = [item for item in res]
rdr2_tweets_df = pd.DataFrame(res_list)

In [83]:
rdr2_tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22043 entries, 0 to 22042
Data columns (total 8 columns):
_id               22043 non-null object
created_at        22043 non-null object
favorite_count    22043 non-null int64
retweet_count     22043 non-null int64
screen_name       22043 non-null object
sentiment         22043 non-null float64
subjectivity      22043 non-null float64
text              22043 non-null object
dtypes: float64(2), int64(2), object(4)
memory usage: 1.3+ MB


In [53]:
test_tweets_df = df
test_tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11022 entries, 0 to 11021
Data columns (total 8 columns):
_id               11022 non-null object
created_at        11022 non-null object
favorite_count    11022 non-null int64
retweet_count     11022 non-null int64
screen_name       11022 non-null object
sentiment         11022 non-null float64
subjectivity      11022 non-null float64
text              11022 non-null object
dtypes: float64(2), int64(2), object(4)
memory usage: 689.0+ KB
